<a href="https://colab.research.google.com/github/crerarc/DataScience/blob/main/eurorndmchck2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gambler's Paradox and the Euromillion Lottery

Playground to test various library functionalities using Euromillion data as
random number generator.



# License
Copyright 2022 Crerar Christie <crerarc03@gmail.com>
 
This program is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option) any later version.
This program is distributed in the hope that it will be useful, but WITHOUT
ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
You should have received a copy of the GNU General Public License along with
this program. If not, see <https://www.gnu.org/licenses/>

In [ ]:
#@title Module Imports

# Import libraries
import random
import datetime as dt
import itertools as itr
import os
from numpy import select as npselect
import pandas as pd
#import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots as mk_subplt
from google.colab import drive # Google file access

In [ ]:
#@title MAIN SEQUENCE
def main() -> None:
    """Main run sequence
    """

    # Data file
    drive = "gdrive/My Drive/Colab Notebooks/Data/"
    #ifile = "/mnt/Disk01/DataA/ProjectsA/python_workspace/EuroMilLot/"
    #ifile = "/home/crerar/DataA/ProjectsA/python_workspace/EuroMilLot/"

    # Get data
    df5, df2 = get_data(drive)
    print("Input of current data: successfull")

    # Establish Frequencies and distribution
    ds5_frq, df5_dsb = get_freqs(df5)
    ds2_frq, df2_dsb = get_freqs(df2)

    # Establish Random Frequencies and distributions
    df5r, df2r = get_random_data(len(df5))
    ds5r_frq, df5r_dsb = get_freqs(df5r)
    ds2r_frq, df2r_dsb = get_freqs(df2r)

    # Incorporate random frequency and distribution data into actual and theoretical
    df5_frq = pd.DataFrame({'Actual': ds5_frq, 'Random': ds5r_frq})
    df5_dsb['Random'] = df5r_dsb['Actual'].copy()
    df2_frq = pd.DataFrame({'Actual': ds2_frq, 'Random': ds2r_frq})
    df2_dsb['Random'] = df2r_dsb['Actual'].copy()

    # Plot distributions
    fn_ret = dstrb_plot(df5_frq, df5_dsb, df2_frq, df2_dsb)

    # Check Random each time vs same each time
    fn_ret = pick_pref(df5, df2, df5r, df2r)

    # Get frequent picks
    frqnt_f5, frqnt_f2 = draw_stats(df5_frq, df5_dsb, df2_frq, df2_dsb)

    # This block establishes statistics and sweetspot for ratio of frequently to infrequently
    # drawn numbers and sums
    # # Create new df columns with no of frequent nos chosed and if sum is frequent
    # df5['nif'] = df5.apply(
    #     lambda row: (row.B1 in frqnt_f5) + (row.B2 in frqnt_f5) + (row.B3 in frqnt_f5) + \
    #                 (row.B4 in frqnt_f5) + (row.B5 in frqnt_f5), axis = 1)
    # df2['nif'] = df2.apply(lambda row: (row.S1 in frqnt_f2) + (row.S2 in frqnt_f2), axis = 1)
    # print(df5['nif'].describe())
    # print(df2['nif'].describe())

    fn_ret = get_guess(frqnt_f5, frqnt_f2, df5_dsb, df2_dsb)

    return 0 # Return successful completion = zero errors, or errors = False

In [ ]:
#@title Function: "get_data" - Reads draw results and update repository as reqd
def get_data(rt_dir: str) -> tuple:
    """[summary]

    Args:
        ifile (str): input repository directory

    Returns:
        tuple: df1, df2 - dataframes for 5 main balls and 2 star balls respectively
    """

    # COLLAB for user repository file
    # ...Check path open
    if not(os.path.ismount("/content/gdrive")):
        drive.mount("/content/gdrive")
    #print(os.listdir())

    # LOCAL for user repository file
    rfile = 'https://www.national-lottery.co.uk/results/euromillions/draw-history/csv'

    # Create parser to parse date variables
    my_parser = lambda csv_date: dt.datetime.strptime(csv_date, "%d-%b-%Y")

    # Read in historic draws csv as a datafile with parsed date as index
    src_file = rt_dir + "euroLotData.csv"
    df_draws_repo = pd.read_csv(src_file, parse_dates = ['DrawDate'],
                                date_parser = my_parser,
                                index_col = 0).fillna(value = 0)
    #print(df_draws_repo.head())

    # Read in latest draw table csv, with parsed date as index
    df_draws_ltst = pd.read_csv(rfile, parse_dates= ['DrawDate'],
                                date_parser = my_parser,
                                index_col = 0).fillna(value = 0)
    #print(df_draws_ltst.head())

    # Get first entry in repository, and check timestamp
    repo_last = df_draws_repo.index[0]
    ltst_last = df_draws_ltst.index[0]
    if ltst_last > repo_last:
        # Create a dataframe of missing values, aligned with repository
        print("! Repo out of date !")
        print(f"Last Repo: {repo_last.day}, {repo_last.month}, {repo_last.year}")
        print(f"Last Draw: {ltst_last.day}, {ltst_last.month}, {ltst_last.year}")
        print("! Appending !")
        df_2mrg = df_draws_ltst.loc[df_draws_ltst.index > repo_last, :'Lucky Star 2']
        df_2mrg.columns = ['B1', 'B2', 'B3', 'B4', 'B5', 'S1', 'S2']
        # Concatenate existing repo with missing data to move missing to top
        df_drws = pd.concat([df_2mrg, df_draws_repo])
        # Create a new file for the data
        tst_fil = "test_out.csv"
        df_drws.to_csv(tst_fil, date_format="%d-%b-%Y")
        # Provision to prevent over-write of original file 
        !mv "test_out.csv" "gdrive/My Drive/Colab Notebooks/Data/euroLotData.csv"
        # mvfile = 'mv ' + ifile + 'test_out.csv ' + lfile
        # print(mvfile)
        # os.system(mvfile)
    else:
        print("No change to Repository Required")
        df_drws = df_draws_repo.copy()

    # Split data frame for 5 draw and 2 draw parts and sum the outcomes
    df1 = df_drws.iloc[:,:-2].astype('int')
    df1["Sum"] = df1.sum(axis = 1).astype('int')
    df2 = df_drws.iloc[:,-2:].astype('int')
    df2["Sum"] = df2.sum(axis = 1).astype('int')

    return df1, df2


In [ ]:
#@title Function: "get freqs"
def get_freqs(data_frame: pd.DataFrame) -> tuple:
    """For supplied dataframe, calc how mamy times each number has occurred
        and log the independent ways in which a sum of the draw has been found

    Args:
        data_frame (pd.DataFrame): Supplied dataframe

    Returns:
        tuple: [description]
    """

    # No of draws
    no_of_draws = len(data_frame)
    drawn_balls = len(data_frame.columns) - 1 # Should be 5 or 2
    no_ball_draws = no_of_draws * drawn_balls

    # Individual Number frequencies
    if drawn_balls == 5:
        e_str = 'B5'
    else:
        e_str = 'S2'
    number_freq_ds = pd.Series(data_frame.loc[:, :e_str].\
                                apply(pd.value_counts).sum(axis = 1)).div(no_ball_draws)
    #number_freq_ds = pd.Series(data_frame.loc[:, :e_str].\
    #                           apply(pd.value_counts).sum(axis = 1).astype('int'))
    # srs_nofrqs = data_frame[['B1', 'B2', 'B3', 'B4', 'B5']].\
    #                           apply(pd.Series.value_counts) # By the column
    # print(number_freq_ds.head())

    # Distribution of summed draws
    # Theoretically, how many independent ways are there to achieve summed value of drawn balls
    mx_ball_no = number_freq_ds.index.max()
    # print(f'Max ball no = {mx_ball_no}')
    # Find all combinations of drawn_balls, from a range of balls numbered from 1 to max ball number
    comb_lst = list(itr.combinations(range(1, mx_ball_no + 1), drawn_balls))
    ncomb = len(comb_lst)
    # Sum each combination and add it to a dictionary with the sum of the combination as key
    sum_freq_dic = {}
    for comb in comb_lst:
        sum_int = sum(comb)
        if sum_int in sum_freq_dic:
            sum_freq_dic[sum_int].append(comb)
        else:
            sum_freq_dic[sum_int] = [comb]
    # For each sum value, establish how many ways (independent draws) there are to achive sum
    no_of_ways_dic = {k:len(v) for (k,v) in sum_freq_dic.items()}
    # Make dictionary a series, index = sum of drawn balls, value = no. of ways
    distrib_theory_ds = pd.Series(no_of_ways_dic) #, index = no_of_ways_dic.keys())

    # Actually, get sum distribution  - Note, assumes no repeated draws
    distrib_actual_ds = data_frame['Sum'].value_counts()

    # Create a dataframe from dictionary of data series
    # d = {'Theory': distrib_theory_ds, 'Actual': distrib_actual_ds}
    return_dist_df = pd.DataFrame({'Theory': distrib_theory_ds.div(ncomb).round(5),
                            'Actual': distrib_actual_ds.div(no_of_draws).round(5)}).fillna(0)

    return number_freq_ds, return_dist_df


In [ ]:
#@title Function: "get_random_data"
def get_random_data(rnd_rows: int) -> tuple:
    """Generate rnd_rows of random draws as a comparison

    Args:
        rnd_rows (int): Amount of random numbers to generate

    Returns:
        tuple: two dataframes:
                r5_df - Random 5 ball draws
                r2_df - Random 2 ball draws
    """
    random.seed(dt.datetime.now())
    rnd5_dct = {k:sorted(random.sample(range(1,51), 5)) for k in range(rnd_rows)}
    r5_df = pd.DataFrame.from_dict(rnd5_dct, orient = 'index',
                                    columns = ['B1', 'B2', 'B3', 'B4', 'B5'])
    r5_df["Sum"] = r5_df.sum(axis = 1).astype('int')

    rnd2_dct = {k:sorted(random.sample(range(1,13), 2)) for k in range(rnd_rows)}
    r2_df = pd.DataFrame.from_dict(rnd2_dct, orient = 'index', columns = ['S1', 'S2'])
    r2_df["Sum"] = r2_df.sum(axis = 1).astype('int')

    return r5_df, r2_df


In [ ]:
#@title Function: "dstrb_plot"
def dstrb_plot(frq5_df: pd.DataFrame, dst5_df: pd.DataFrame,
                frq2_df: pd.DataFrame, dst2_df: pd.DataFrame) -> int:
    """Plot distributions from dataframe

    Args:
        dstrb_df (pd.DataFrame): Target dataframe
        freqs_ds (pd.Series) : Draw frequencies

    Returns:
        int: Nominal return value
    """

    # ...Setup straight lines thru theoretical values
    xp5 = [1, 50]
    yp5 = [0.02, 0.02]
    xp2 = [1, 12]
    yp2 = [0.0833, 0.0833]

    # Plotly graph object plotting method
    #fig = go.FigureWidget()
    fig = mk_subplt(rows = 2, cols = 2)

    # Top Left - Frequency 5 draw
    fig.add_trace(go.Scatter(x = frq5_df.index, y = frq5_df['Random'],
                             name = 'Drw5:Frq:R', line = dict(color = '#94c973')),
                             row = 1, col = 1)
    fig.add_trace(go.Scatter(x = frq5_df.index, y = frq5_df['Actual'],
                            name = 'Drw5:Frq:A', line = dict(color = '#043bb1')),
                            row = 1, col = 1)
    fig.add_trace(go.Scatter(x = xp5, y = yp5, name = 'Drw5:Frq:T',
                            line = dict(color = '#ff0000')), row = 1, col = 1)

    # Top Right - Frequency 2 draw
    fig.add_trace(go.Scatter(x = frq2_df.index, y = frq2_df['Random'],
                             name = 'Drw2:Frq:R', line = dict(color = '#94c973')),
                             row = 1, col = 2)
    fig.add_trace(go.Scatter(x = frq2_df.index, y = frq2_df['Actual'],
                            name = 'Drw2:Frq:A', line = dict(color = '#043bb1')),
                            row = 1, col = 2)
    fig.add_trace(go.Scatter(x = xp2, y = yp2, name = 'Drw2:Frq:T',
                            line = dict(color = '#ff0000')), row = 1, col = 2)

    # Btm Left - Distrib 5
    fig.add_trace(go.Scatter(x = dst5_df.index, y = dst5_df['Random'],
                            name = 'Drw5:Dst:R', line = dict(color = '#94c973')),
                            row = 2, col = 1)
    fig.add_trace(go.Scatter(x = dst5_df.index, y = dst5_df['Actual'],
                            name = 'Drw5:Dst:A', line = dict(color = '#043bb1')),
                            row = 2, col = 1)
    fig.add_trace(go.Scatter(x = dst5_df.index, y = dst5_df['Theory'],
                            name = 'Drw5:Dst:T', line = dict(color = '#ff0000')),
                            row = 2, col = 1)

    # Btm Right - Distrib 2
    fig.add_trace(go.Scatter(x = dst2_df.index, y = dst2_df['Random'],
                            name = 'Drw2:Dst:R', line = dict(color = '#94c973')),
                            row = 2, col = 2)
    fig.add_trace(go.Scatter(x = dst2_df.index, y = dst2_df['Actual'],
                            name = 'Drw2:Dst:A', line = dict(color = '#043bb1')),
                            row = 2, col = 2)
    fig.add_trace(go.Scatter(x = dst2_df.index, y = dst2_df['Theory'],
                            name = 'Drw2:Dst:T', line = dict(color = '#ff0000')),
                            row = 2, col = 2)

    # Single plot with plotly graphic objects
    # fig.add_scatter(x = dstrb_df.index, y = dstrb_df['Theory'], name = 'Theory')
    # fig.update_xaxes(title = "Sum")
    # fig.update_yaxes(title = "Probability of draw")


    # Plotly Express plotting method
    # fig = px.line(dstrb_df, x = dstrb_df.index, y = 'Actual') # Nope

    # plt_df_a = pd.DataFrame({"Sum": dstrb_df.index,
    #                       "Type": "Actual",
    #                       "Values": dstrb_df['Actual']})
    # plt_df_b = pd.DataFrame({"Sum": dstrb_df.index,
    #                       "Type": "Theory",
    #                       "Values": dstrb_df['Theory']})
    # plt_df = pd.concat([plt_df_a, plt_df_b])
    # fig = px.line(plt_df, x= "Sum", y = "Values", color = 'Type')
    # print(plt_df_b.head())

    fig.show()

    return 0


In [ ]:
#@title Function: "prize_rank" - Ranking of winning matches
def prize_rank(drawn5: list, drawn2: list, pickd5: list, pickd2: list) -> int:
    """Return a prize rank for drawn numbers

    Args:
        drawn5 (list): 5 drawn to check against
        drawn2 (list): 2 drawn to check against
        pickd5 (list): 5 drawn to check
        pickd2 (list): 2 drawn to check

    Returns:
        int: Prize ranking

    Method: Set intersection to count no in intersection
        13 chances to win:
        Default no win rank: 14
        Idx   5 & 2 Match Rank
        [0]     2 + 0       13
        [1]     2 + 1       12
        [2]     1 + 2       11
        [3]     3 + 0       10
        [4]     3 + 1       9
        [5]     2 + 2       8
        [6]     4 + 0       7
        [7]     3 + 2       6
        [8]     4 + 1       5
        [9]     4 + 2       4
        [10]    5 + 0       3
        [11]    5 + 1       2
        [12]    5 + 2       1
    """

    # Find all the matching numbers
    mtch5 = set(drawn5).intersection(set(pickd5))
    mtch2 = set(drawn2).intersection(set(pickd2))

    # Convert no of wins into 5&2 match result
    res = str(len(mtch5)) + str(len(mtch2))

    # Calculate the rank of the win from no of matching balls in 5 and 2 draws
    ranks = {'52':1, '51':2, '50':3, '42':4, '41':5, '32': 6, '40':7,
            '22':8, '31':9, '30':10, '12':11, '21':12, '20':13}

    # Return prize rank
    if res in ranks:
        return ranks[res]
    else:
        return 14


In [ ]:
#@title Function: "draw_stats" - Find a way to make better predictions
def draw_stats(frq5_df: pd.DataFrame, dst5_df: pd.DataFrame,
              frq2_df: pd.DataFrame, dst2_df: pd.DataFrame) -> tuple:
    """Summarise differences between Theory and Actual and theory and Random

    Args:
        frq5_df (pd.DataFrame): Frequency of numbers for 5 draw
        dst5_df (pd.DataFrame): Distribution of sums for 5 draw
        frq2_df (pd.DataFrame): Frequency of numbers for 2 draw
        dst2_df (pd.DataFrame): Distribution of sums for 2 draw

    Returns:
        frqnt_f5 (list): Ball numbers within 25%/75% frequency limits
        frqnt_d5 (list): Draw sums within 25%/75% frequency limits
        frqnt_f2 (list): Ball numbers within 25%/75% frequency limits
        frqnt_d2 (list): Draw sums within 25%/75% frequency limits
        drw_Res (pd.DataFrame): Description stats
    """

    # Deviations from Theoretical
    frq5_df['DevA'] = frq5_df['Actual'] - (1/50)
    frq5_df['DevR'] = frq5_df['Random'] - (1/50)
    frq2_df['DevA'] = frq2_df['Actual'] - (1/12)
    frq2_df['DevR'] = frq2_df['Random'] - (1/12)

    dst5_df['DevA'] = dst5_df['Actual'] - dst5_df['Theory']
    dst5_df['DevR'] = dst5_df['Random'] - dst5_df['Theory']
    dst2_df['DevA'] = dst2_df['Actual'] - dst2_df['Theory']
    dst2_df['DevR'] = dst2_df['Random'] - dst2_df['Theory']
    # print(frq5_df.sort_values(['DevA']))

    drw_res = pd.DataFrame({'FrqA5': frq5_df['DevA'].describe(),
                            'FrqR5': frq5_df['DevR'].describe(),
                            'FrqA2': frq2_df['DevA'].describe(),
                            'FrqR2': frq2_df['DevR'].describe(),
                            'DstA5': dst5_df['DevA'].describe(),
                            'DstR5': dst5_df['DevR'].describe(),
                            'DstA2': dst2_df['DevA'].describe(),
                            'DstR2': dst2_df['DevR'].describe()})

    # Comparative deviations for Actual and Random against theoretical
    drw_res['F5:A-R'] = drw_res['FrqA5'] - drw_res['FrqR5']
    drw_res['D5:A-R'] = drw_res['DstA5'] - drw_res['DstR5']
    drw_res['F2:A-R'] = drw_res['FrqA2'] - drw_res['FrqR2']
    drw_res['D2:A-R'] = drw_res['DstA2'] - drw_res['DstR2']
    #print(drw_res[['F5:A-R', 'D5:A-R', 'F2:A-R', 'D2:A-R']].multiply(10000).astype(int))

    # print(frq5_df.loc[frq5_df['DevA'] == frq5_df['DevA'].min()])
   # print('Number frequencies')
   # print(frq5_df[(frq5_df['DevA'] > drw_res.at['75%', 'FrqA5']) |
   #               (frq5_df['DevA'] < drw_res.at['25%', 'FrqA5'])])
   # print(dst5_df[(dst5_df['DevA'] > drw_res.at['75%', 'DstA5']) |
   #               (dst5_df['DevA'] < drw_res.at['25%', 'DstA5'])])

    # Numbers inside and outside central band
    print("Calculating Frequency Sets")
    conditions = [(frq5_df['DevA'] > drw_res.at['75%', 'FrqA5']) |
                  (frq5_df['DevA'] < drw_res.at['25%', 'FrqA5']),
                  (frq5_df['DevA'] <= drw_res.at['75%', 'FrqA5']) &
                  (frq5_df['DevA'] >= drw_res.at['25%', 'FrqA5'])]
    vals = ['Out', 'In']
    frq5_df['Frequent'] = npselect(conditions, vals)
    conditions = [(frq2_df['DevA'] > drw_res.at['75%', 'FrqA2']) |
                  (frq2_df['DevA'] < drw_res.at['25%', 'FrqA2']),
                  (frq2_df['DevA'] <= drw_res.at['75%', 'FrqA2']) &
                  (frq2_df['DevA'] >= drw_res.at['25%', 'FrqA2'])]
    frq2_df['Frequent'] = npselect(conditions, vals)
    conditions = [(dst5_df['DevA'] > drw_res.at['75%', 'DstA5']) |
                  (dst5_df['DevA'] < drw_res.at['25%', 'DstA5']),
                  (dst5_df['DevA'] <= drw_res.at['75%', 'DstA5']) &
                  (dst5_df['DevA'] >= drw_res.at['25%', 'DstA5'])]
    dst5_df['Frequent'] = npselect(conditions, vals)
    conditions = [(dst2_df['DevA'] > drw_res.at['75%', 'DstA2']) |
                  (dst2_df['DevA'] < drw_res.at['25%', 'DstA2']),
                  (dst2_df['DevA'] <= drw_res.at['75%', 'DstA2']) &
                  (dst2_df['DevA'] >= drw_res.at['25%', 'DstA2'])]
    dst2_df['Frequent'] = npselect(conditions, vals)

    # print(frq5_df[(frq5_df['DevA'] > drw_res.at['75%', 'FrqA5']) |
    #               (frq5_df['DevA'] < drw_res.at['25%', 'FrqA5'])])
    frqnt_f5 = frq5_df.index[frq5_df['Frequent'] == 'In'].tolist()
    # frqnt_d5 = dst5_df.index[dst5_df['Frequent'] == 'In'].tolist()
    frqnt_f2 = frq2_df.index[frq2_df['Frequent'] == 'In'].tolist()
    # frqnt_d2 = dst2_df.index[dst2_df['Frequent'] == 'In'].tolist()

    # Totals inside and outside central band
    return (frqnt_f5, frqnt_f2)


In [ ]:
#@title Get a best guess
def get_guess(frq5: pd.DataFrame, frq2: pd.DataFrame,
            dsb5: pd.DataFrame, dsb2: pd.DataFrame) -> int:
    """[summary]

    Args:
        frq5 (pd.DataFrame): [Frequencies for Draw 5 numbers]
        frq2 (pd.DataFrame): [Frequencies for Draw 2 numbers]
        dsb5 (pd.DataFrame): [Distributions for Draw 5 numbers]
        dsb2 (pd.DataFrame): [Distributions for Draw 2 numbers]

    Returns:
        guess_5: list of main_balls to choose
        guess_2: list of star balls to choose
    """
    # Results - 3 - 4 frequent numbers in 5 draw, at least one freqent in 2 draw
    good_guess = False
    combs = {}
    while not good_guess:
        guess5 = sorted(random.sample(range(1,51), 5))
        no_in = len(set(frq5).intersection(set(guess5)))
        if no_in >= 2 and no_in <= 4:
            good_guess = True
    good_guess = False
    while not good_guess:
        guess2 = sorted(random.sample(range(1,13), 2))
        no_in = len(set(frq2).intersection(set(guess2)))
        if no_in >=1:
            good_guess = True
    guess5_sum = sum(guess5)
    guess2_sum = sum(guess2)
    print()
    print(f'Guess is: {guess5}{guess2}')
    print(f'Sum5: {guess5_sum} | Sum2: {guess2_sum}')
    print(dsb5.loc[guess5_sum : guess5_sum])
    print(dsb2.loc[guess2_sum : guess2_sum])

    return 0


In [ ]:
#@title Function: "pick_pref" - Better to keep one or change each time?
def pick_pref(drw5: pd.DataFrame, drw2: pd.DataFrame,
            drw5r: pd.DataFrame, drw2r: pd.DataFrame) -> list:
    """Check if it's better to choose new random each time, or,
       just keep the same number

    Args:
        drw5 (pd.DataFrame): Actual drawn 5 balls
        drw2 (pd.DataFrame): Actual drawn 2 balls
        drw5r (pd.DataFrame): Random drawn 5 balls
        drw2r (pd.DataFrame): Random drawn 2 balls

    Returns:
        list: [description]
    """

    # Pick New each time
    win_new = []
    drw5_list = drw5.values.tolist()
    drw2_list = drw2.values.tolist()
    drw5r_list = drw5r.values.tolist()
    drw2r_list = drw2r.values.tolist()

    # New each time
    for i, drwl5 in enumerate(drw5_list):
        prize = prize_rank(drwl5, drw2_list[i], drw5r_list[i], drw2r_list[i])
        if prize == 0:
            print(drw5, drw2)
            print(drw5r,drw2r)
        if prize < 14:
            win_new.append(prize)
    print("\nNew each time")
    print(f"There were {len(win_new)} wins from {len(drw5_list)} draws")
    print(f"Prize rank, Max: {min(win_new)} Min: {max(win_new)}")
    #print("Prize breakdown:")
    #for i in range(1,14):
    #    print(f"{i} - {win_new.count(i)}")

    # Keep the same number
    j = random.randint(0, len(drw5_list))
    for i, drwl5 in enumerate(drw5_list):
        prize = prize_rank(drwl5, drw2_list[i], drw5r_list[j], drw2r_list[j])
        if prize == 0:
            print(drw5, drw2)
            print(drw5r,drw2r)
        if prize < 14:
            win_new.append(prize)
    print("\nKeep same numbers")
    print(f"There were {len(win_new)} wins from {len(drw5_list)} draws")
    print(f"Prize rank, Max: {min(win_new)} Min: {max(win_new)}")

    #print("Prize breakdown:")
    #for i in range(1,14):
    #    print(f"{i} - {win_new.count(i)}")

    return 0


In [ ]:
#@title Execution Starting Point
if __name__ == "__main__":
    main()


No change to Repository Required
Input of current data: successfull



New each time
There were 125 wins from 1511 draws
Prize rank, Max: 8 Min: 13

Keep same numbers
There were 270 wins from 1511 draws
Prize rank, Max: 8 Min: 13
Calculating Frequency Sets

Guess is: [1, 13, 17, 24, 39][6, 10]
Sum5: 94 | Sum2: 16
     Theory   Actual  Random     DevA     DevR Frequent
94  0.00736  0.00529  0.0086 -0.00207  0.00124      Out
     Theory   Actual  Random     DevA     DevR Frequent
16  0.06061  0.04103  0.0675 -0.01958  0.00689      Out
